In [3]:
import pandas as pd


In [4]:
enrol = pd.read_csv("../data/processed/enrolment_full.csv")
demo = pd.read_csv("../data/processed/demographic_full.csv")
bio = pd.read_csv("../data/processed/biometric_full.csv")

print("Enrolment shape:", enrol.shape)
print("Demographic shape:", demo.shape)
print("Biometric shape:", bio.shape)


Enrolment shape: (1006029, 7)
Demographic shape: (2071700, 6)
Biometric shape: (1861108, 6)


In [5]:
print("\nMissing values in Enrolment:\n", enrol.isnull().sum())
print("\nMissing values in Demographic:\n", demo.isnull().sum())
print("\nMissing values in Biometric:\n", bio.isnull().sum())



Missing values in Enrolment:
 date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64

Missing values in Demographic:
 date             0
state            0
district         0
pincode          0
demo_age_5_17    0
demo_age_17_     0
dtype: int64

Missing values in Biometric:
 date            0
state           0
district        0
pincode         0
bio_age_5_17    0
bio_age_17_     0
dtype: int64


In [6]:
for df in [enrol, demo, bio]:
    # Fix date format (DD-MM-YYYY)
    df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
    
    # Clean text columns
    df['state'] = df['state'].astype(str).str.title().str.strip()
    df['district'] = df['district'].astype(str).str.title().str.strip()


In [7]:
enrol = enrol.dropna(subset=['date'])
demo = demo.dropna(subset=['date'])
bio = bio.dropna(subset=['date'])

enrol.drop_duplicates(inplace=True)
demo.drop_duplicates(inplace=True)
bio.drop_duplicates(inplace=True)

print("After cleaning:")
print("Enrolment:", enrol.shape)
print("Demographic:", demo.shape)
print("Biometric:", bio.shape)


After cleaning:
Enrolment: (983000, 7)
Demographic: (1598012, 6)
Biometric: (1766159, 6)


In [8]:
age_cols_enrol = ['age_0_5', 'age_5_17', 'age_18_greater']
demo_cols = ['demo_age_5_17', 'demo_age_17_']
bio_cols = ['bio_age_5_17', 'bio_age_17_']

for col in age_cols_enrol:
    enrol[col] = pd.to_numeric(enrol[col], errors='coerce').fillna(0)

for col in demo_cols:
    demo[col] = pd.to_numeric(demo[col], errors='coerce').fillna(0)

for col in bio_cols:
    bio[col] = pd.to_numeric(bio[col], errors='coerce').fillna(0)


In [9]:
enrol['total_enrol'] = enrol[age_cols_enrol].sum(axis=1)
demo['total_demo_updates'] = demo[demo_cols].sum(axis=1)
bio['total_bio_updates'] = bio[bio_cols].sum(axis=1)


In [10]:
enrol.to_csv("../data/processed/enrolment_clean.csv", index=False)
demo.to_csv("../data/processed/demographic_clean.csv", index=False)
bio.to_csv("../data/processed/biometric_clean.csv", index=False)

print("Cleaned datasets saved successfully!")


Cleaned datasets saved successfully!


In [1]:
import pandas as pd
import os


path_prefix = "../data/processed/"

# Verify files exist before loading
files_to_check = ["enrolment_clean.csv", "demographic_clean.csv", "biometric_clean.csv"]
for file in files_to_check:
    if not os.path.exists(os.path.join(path_prefix, file)):
        print(f"❌ Error: {file} not found in {path_prefix}")
    else:
        print(f"✅ Found: {file}")

# STEP 2: LOAD CLEANED DATA 
enrol = pd.read_csv(f"{path_prefix}enrolment_clean.csv")
demo = pd.read_csv(f"{path_prefix}demographic_clean.csv")
bio = pd.read_csv(f"{path_prefix}biometric_clean.csv")

# STEP 3: DATE CONVERSION & MONTHLY AGGREGATION 
for df in [enrol, demo, bio]:
    # Ensure date is in datetime format
    df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
    # Create month column for Power BI time-series filtering
    df['month'] = df['date'].dt.to_period('M').astype(str)

# Group by state, district, and month to keep the file size manageable
enrol_agg = enrol.groupby(['state', 'district', 'month'])[['total_enrol']].sum().reset_index()
demo_agg = demo.groupby(['state', 'district', 'month'])[['total_demo_updates']].sum().reset_index()
bio_agg = bio.groupby(['state', 'district', 'month'])[['total_bio_updates']].sum().reset_index()

#STEP 4: MERGE DATASETS
master = enrol_agg.merge(demo_agg, on=['state', 'district', 'month'], how='outer').fillna(0)
master = master.merge(bio_agg, on=['state', 'district', 'month'], how='outer').fillna(0)

#STEP 5: CALCULATE PRIZE-WINNING METRICS 
master['total_updates'] = master['total_demo_updates'] + master['total_bio_updates']
# Service Ratio: High numbers indicate districts with heavy update pressure
master['service_ratio'] = master['total_updates'] / (master['total_enrol'] + 1)

# STEP 6: SAVE FINAL MASTER FILE 
output_path = f"{path_prefix}master_aadhaar_data.csv"
master.to_csv(output_path, index=False)

print("-" * 30)
print(f"✅ SUCCESS! Master file created at: {output_path}")
print(f"Total rows for Power BI: {len(master)}")

✅ Found: enrolment_clean.csv
✅ Found: demographic_clean.csv
✅ Found: biometric_clean.csv
------------------------------
✅ SUCCESS! Master file created at: ../data/processed/master_aadhaar_data.csv
Total rows for Power BI: 13007


In [2]:
import pandas as pd

# Load the file you just created
df = pd.read_csv("../data/processed/master_aadhaar_data.csv")

# 1. Remove noise rows (numeric names)
df = df[~df['state'].str.contains('100000', na=False)]

# 2. Standardize State Names (Mapping duplicates)
state_map = {
    "Andaman & Nicobar Islands": "Andaman And Nicobar Islands",
    "Orissa": "Odisha",
    "West Bangal": "West Bengal",
    "West  Bengal": "West Bengal",
    "West Bengli": "West Bengal",
    "Westbengal": "West Bengal",
    "Chhatisgarh": "Chhattisgarh",
    "Uttaranchal": "Uttarakhand",
    "Pondicherry": "Puducherry"
}
df['state'] = df['state'].replace(state_map)

# 3. Clean up extra spaces in names
df['state'] = df['state'].str.strip()
df['district'] = df['district'].str.strip()

# 4. Handle NaT in months
df['month'] = df['month'].fillna("Unknown")

# 5. Filter out any rows that are completely zero (no activity)
df = df[df['total_updates'] + df['total_enrol'] > 0]

# Save the FINAL polished version
df.to_csv("../data/processed/final_aadhaar_dashboard_data.csv", index=False)

print("🚀 FINAL CLEANING COMPLETE!")
print(f"Unique States remaining: {df['state'].nunique()}")

🚀 FINAL CLEANING COMPLETE!
Unique States remaining: 50
